In [5]:
import pandas as pd 

file_path = "C:/Users/rtp09/final_settlement_prices_complete0320.xlsx"
df = pd.read_excel(file_path)

In [6]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df.dropna(subset=["date"], inplace=True)
df.set_index("date", inplace=True)

price_cols = [col for col in df.columns if "Settlement Price" in col]
symbolic_labels = [col for col in df.columns if col not in price_cols]
column_label_map = dict(zip(price_cols, symbolic_labels))

def parse_settlement_price(cell):
    if pd.isna(cell):
        return []
    return [float(x.strip()) for x in str(cell).split(',')]

# Implement Panama adjustment
adjusted_series = {}

for price_col, label in column_label_map.items():
    prices = df[price_col].apply(parse_settlement_price)
    raw_series = pd.Series(index=prices.index, dtype='float64')

    for i, entry in enumerate(prices):
        if not entry:
            raw_series.iloc[i] = None
        else:
            raw_series.iloc[i] = entry[0]  # old contract (even on roll day)

    adjusted_series_final = raw_series.copy()
    for i, (date, entry) in enumerate(prices.items()):
        if entry is not None and len(entry) == 2:
            old_price, new_price = entry
            roll_gap = new_price - old_price
            adjusted_series_final.iloc[:i+1] += roll_gap  # Apply to all prior values including roll day

    adjusted_series[label] = adjusted_series_final

adjusted_df = pd.DataFrame(adjusted_series)

In [7]:
adjusted_df

,SR1.c.0,SR1.c.1,SR1.c.2,SR1.c.3,SR1.c.4,1F,2F,3F,4F,5F,6F,7F,8F,9F,10F,11F,12F,13F
date,,,,,,,,,,,,,,,,,,
2024-03-18,95.6300,95.5825,95.795,95.970,96.065,95.9125,96.055,96.055,96.080,96.135,96.190,96.210,96.210,96.215,96.215,96.220,96.220,96.220
2024-03-19,95.6300,95.5825,95.795,95.980,96.075,95.9125,96.075,96.075,96.110,96.180,96.250,96.280,96.280,96.285,96.280,96.280,96.275,96.275
2024-03-20,95.6300,95.5825,95.815,96.015,96.125,95.9225,96.130,96.165,96.215,96.285,96.350,96.365,96.350,96.345,96.335,96.330,96.315,96.310
2024-03-21,95.6300,95.5825,95.810,96.005,96.115,95.9175,96.110,96.150,96.190,96.255,96.315,96.335,96.335,96.335,96.325,96.320,96.310,96.305
2024-03-22,95.6300,95.5825,95.815,96.020,96.130,95.9225,96.130,96.170,96.215,96.285,96.355,96.385,96.390,96.395,96.390,96.385,96.370,96.360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-13,95.6725,95.6900,95.765,95.845,95.945,95.7075,95.960,96.210,96.360,96.450,96.490,96.495,96.480,96.455,96.425,96.395,96.365,96.330
2025-03-14,95.6750,95.6900,95.755,95.825,95.925,95.7000,95.930,96.145,96.280,96.365,96.405,96.415,96.410,96.390,96.365,96.340,96.310,96.275
2025-03-17,95.6750,95.6900,95.740,95.800,95.885,95.6875,95.885,96.095,96.235,96.325,96.375,96.390,96.385,96.370,96.350,96.325,96.300,96.265


In [4]:
output_path = "C:/Users/rtp09/back_adjusted_sofr_rates0320.csv"
adjusted_df.to_csv(output_path)